<a href="https://colab.research.google.com/github/maheravi/Deep-Learning/blob/main/Recurrent%20Neural%20Network/Joon_Del_Update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
import cv2
import os
from tensorflow.keras.layers import Conv2D, MaxPooling2D, SimpleRNN, GRU, LSTM, Dense, Flatten, TimeDistributed
from matplotlib import pyplot as plt

In [20]:
batch_size = 12
epoch = 10
lr = 0.001
width = height = 50

In [21]:
path = "/content/drive/MyDrive/joon_del/"
dirs = os.listdir(path)
dirs

['0', '1']

In [22]:
path = "/content/drive/MyDrive/joon_del/"
dirs = os.listdir(path)

File=[]
label=[]

for dir in dirs:
  subfolder = f"/content/drive/MyDrive/joon_del/{dir}"
  files = os.listdir(subfolder)
  for f in files:
    cap = cv2.VideoCapture(f'/content/drive/MyDrive/joon_del/{dir}/{f}')
    video=[]
    while(True):
      ret, frame = cap.read()
      if ret == False:
          break
      frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      img = cv2.resize(frame, dsize=(width, height))
      img = img / 255.
      video.append(img)
    File.append(video)
    label.append(float(dir))




In [23]:
longest_list = max(len(f) for f in File)
longest_list

313

In [24]:
for i, f in enumerate(File):
  if len (f) < longest_list:
    for j in range(longest_list-len(f)):
      File[i].append(np.zeros((width, height, 3)))

In [25]:
len(File)

68

In [26]:
newFile = np.array(File)
label = np.array(label)
label = label[..., np.newaxis]

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(newFile, label, test_size=0.2, random_state=42)

In [28]:
print('train data size:', X_train.shape)
print('train label size:', y_train.shape)


train data size: (54, 313, 50, 50, 3)
train label size: (54, 1)


In [29]:
crnn = tf.keras.models.Sequential([
                  # CNN
                  TimeDistributed(Conv2D(16, (3, 3), activation="relu", input_shape=(None, width, height, 3))),
                  TimeDistributed(MaxPooling2D(pool_size=(2, 2))),

                  TimeDistributed(Flatten()),

                  # RNN
                  SimpleRNN(50, return_sequences=True),
                  SimpleRNN(20),
                  Dense(2, activation="softmax")
])

In [30]:
crnn.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])

crnn.fit(X_train, y_train, batch_size=batch_size, epochs=epoch)

Epoch 1/10
5/5 [==============================] - 9s 1s/step - loss: 0.7511 - accuracy: 0.5185
Epoch 2/10
5/5 [==============================] - 6s 1s/step - loss: 0.6312 - accuracy: 0.7593
Epoch 3/10
5/5 [==============================] - 7s 1s/step - loss: 0.8721 - accuracy: 0.3148
Epoch 4/10
5/5 [==============================] - 6s 1s/step - loss: 0.7566 - accuracy: 0.4259
Epoch 5/10
5/5 [==============================] - 6s 1s/step - loss: 0.7430 - accuracy: 0.5185
Epoch 6/10
5/5 [==============================] - 6s 1s/step - loss: 0.6940 - accuracy: 0.5926
Epoch 7/10
5/5 [==============================] - 6s 1s/step - loss: 0.7414 - accuracy: 0.5370
Epoch 8/10
5/5 [==============================] - 6s 1s/step - loss: 0.6902 - accuracy: 0.5556
Epoch 9/10
5/5 [==============================] - 6s 1s/step - loss: 0.6945 - accuracy: 0.6111
Epoch 10/10
5/5 [==============================] - 6s 1s/step - loss: 0.7772 - accuracy: 0.4630


In [18]:
crnn_eval = crnn.evaluate(X_test, y_test)


1/1 [==============================] - 1s 643ms/step - loss: 0.8415 - accuracy: 0.2143


#Inference

In [43]:
cap = cv2.VideoCapture(f'/content/drive/MyDrive/joon_del/1/012.mp4')
video=[]
while(True):
  ret, frame = cap.read()
  if ret == False:
      break
  frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
  img = cv2.resize(frame, dsize=(width, height))
  img = img / 255.
  video.append(img)


In [44]:
if len (video) < longest_list:
    for j in range(longest_list-len(video)):
      video.append(np.zeros((width, height, 3)))

In [45]:
Video = np.array(video)
Video = Video[np.newaxis, ...]

In [46]:
pred = crnn.predict(Video)
prediction = np.argmax(pred)
if prediction == 1:
  print('Khodeshe')
else:
  print('khodesh nist')

Khodeshe
